In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Establish connection with Postgres Database
rds_connection_string = "postgres:postgres@localhost:5432/ETLProject"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
#Create dataframes with raw data from csv files 
stations_df = pd.read_csv("Resources/stations.csv")
lines_df = pd.read_csv("Resources/lines.csv")
stationlines_df = pd.read_csv("Resources/station_lines.csv")
tracks_df = pd.read_csv("Resources/tracks.csv")
tracklines_df = pd.read_csv("Resources/track_lines.csv")
cities_df = pd.read_csv("Resources/cities.csv")
systems_df = pd.read_csv("Resources/systems.csv")

In [4]:
#Verify the counts before duplicate removal
count_df = [stations_df['id'].count(), \
            lines_df['id'].count(), \
            stationlines_df['id'].count(), \
            tracks_df['id'].count(), \
            tracklines_df['id'].count(), \
            cities_df['id'].count(), \
            systems_df['id'].count()
           ]
count_df

[15794, 1343, 16242, 9271, 9757, 334, 488]

In [5]:
#Remove Duplicates
stations_df = stations_df.drop_duplicates (subset=['name', 'geometry', 'buildstart', 'opening', 'closure', 'city_id'], keep='first', inplace=False)
lines_df = lines_df.drop_duplicates (subset=['name', 'system_id', 'city_id'], keep='first', inplace=False)
stationlines_df = stationlines_df.drop_duplicates (subset=['station_id', 'line_id', 'city_id'], keep='first', inplace=False)
tracks_df = tracks_df.drop_duplicates (subset=['closure', 'length', 'city_id'], keep='first', inplace=False)
tracklines_df = tracklines_df.drop_duplicates (subset=['section_id', 'line_id', 'city_id'], keep='first', inplace=False)
cities_df = cities_df.drop_duplicates (subset=['name', 'coords', 'country'], keep='first', inplace=False)
systems_df = systems_df.drop_duplicates (subset=['city_id'], keep='first', inplace=False)

In [6]:
#Verify the counts after duplicate removal
count_df = [stations_df['id'].count(), \
            lines_df['id'].count(), \
            stationlines_df['id'].count(), \
            tracks_df['id'].count(), \
            tracklines_df['id'].count(), \
            cities_df['id'].count(), \
            systems_df['id'].count()
           ]
count_df

[15792, 1343, 16242, 7744, 9757, 334, 287]

In [7]:
#Split geometry column to understand Latitude and Longitude
stations_df[['Latitude','Longitude']] = stations_df.geometry.str.split(" ",expand=True,)
stations_df['Latitude'] = stations_df['Latitude'].map(lambda x: x.lstrip('POINT('))
stations_df['Longitude'] = stations_df['Longitude'].map(lambda x: x.rstrip(')'))
stations_df.head()

,id,name,geometry,buildstart,opening,closure,city_id,Latitude,Longitude
0,7694,Keisei Tsudanuma,POINT(140.024812197129 35.6837744784723),1921.0,1921.0,999999.0,114,140.024812197129,35.6837744784723
1,6003,Kossuth Lajos tér,POINT(19.0462376564033 47.5054880717671),0.0,0.0,999999.0,29,19.0462376564033,47.5054880717671
2,7732,Saint-Charles,POINT(5.3801556 43.3024646),1973.0,1977.0,999999.0,74,5.3801556,43.3024646
3,7695,Keisei Makuhari-Hongo,POINT(140.042146725175 35.6726021159981),1991.0,1991.0,999999.0,114,140.042146725175,35.6726021159981
4,7726,Chartreux,POINT(5.4014815 43.309129),1973.0,1977.0,999999.0,74,5.4014815,43.309129


In [8]:
#Split coordinates column to understand Latitude and Longitude
cities_df[['Latitude','Longitude']] = cities_df.coords.str.split(" ",expand=True,)
cities_df['Latitude'] = cities_df['Latitude'].map(lambda x: x.lstrip('POINT('))
cities_df['Longitude'] = cities_df['Longitude'].map(lambda x: x.rstrip(')'))
cities_df.head()

,id,name,coords,start_year,url_name,country,country_state,Latitude,Longitude
0,5,Aberdeen,POINT(-2.15 57.15),2017.0,aberdeen,Scotland,NaN,-2.15,57.15
1,6,Adelaide,POINT(138.6 -34.91666667),2017.0,adelaide,Australia,NaN,138.6,-34.91666667
2,7,Algiers,POINT(3 36.83333333),2017.0,algiers,Algeria,NaN,3,36.83333333
3,9,Ankara,POINT(32.91666667 39.91666667),2017.0,ankara,Turkey,NaN,32.91666667,39.91666667
4,16,Belém,POINT(-48.48333333 -1.466666667),2017.0,belem,Brazil,NaN,-48.48333333,-1.466666667


In [9]:
#Picking up the columns only with the existing data
tracks_df = tracks_df[['id', 'closure', 'length', 'city_id']].copy()
tracks_df.head()

,id,closure,length,city_id
0,1911,999999.0,6719,29
1,2563,999999.0,199,118
2,2557,999999.0,925,118
3,2558,999999.0,881,118
4,2564,999999.0,213,118


In [10]:
#Load cleansed data into tables in Postgres with REPLACE option
stations_df.to_sql(name='newstations', con=engine, if_exists='replace', index=False)
lines_df.to_sql(name='lines', con=engine, if_exists='replace', index=False)
stationlines_df.to_sql(name='stationlines', con=engine, if_exists='replace', index=False)
tracks_df.to_sql(name='newtracks', con=engine, if_exists='replace', index=False)
tracklines_df.to_sql(name='tracklines', con=engine, if_exists='replace', index=False)
cities_df.to_sql(name='newcities', con=engine, if_exists='replace', index=False)
systems_df.to_sql(name='systems', con=engine, if_exists='replace', index=False)

In [15]:
#Confirm that data has been added to the tables in Postgres
pd.read_sql_query('select * from newstations', con=engine)['id'].count()

dbcount_df = [pd.read_sql_query('select * from newstations', con=engine)['id'].count(), \
                pd.read_sql_query('select * from lines', con=engine)['id'].count(), \
                pd.read_sql_query('select * from stationlines', con=engine)['id'].count(), \
                pd.read_sql_query('select * from newtracks', con=engine)['id'].count(), \
                pd.read_sql_query('select * from tracklines', con=engine)['id'].count(), \
                pd.read_sql_query('select * from newcities', con=engine)['id'].count(), \
                pd.read_sql_query('select * from systems', con=engine)['id'].count()
           ]
dbcount_df

[15792, 1343, 16242, 7744, 9757, 334, 287]